# Optimising Information Gathering Following A Major Criminal Incident

Mitchell Doody-Burras 23240401

In [ ]:
import networkx as nx
import numpy as np
import math
from scipy.spatial import Voronoi
import matplotlib.pyplot as plt
import matplotlib.animation as animation
%matplotlib widget

## The Agents

The `Patrol` builder.

In [ ]:
class Patrol:
    
    def __init__(self, world, pos, incident, cost):
        """ Initialise patrol attributes. """
        self.world = world
        self.world_pos = pos
        self.incident = incident
        self.pos = np.array((0, 0))
        self.coord = np.array(((0, 0), (0, 0)))
        self.coord_list = np.empty((0, 2))
        self.resolved = False
        self.bank = 0
        self.current_node = np.random.choice(list(world.nodes()))
        self.cost = cost

    def move(self): 
        if self.at_next_node():
            if self.at_incident():
                self.resolved = True
                # need this currently because what if initalised on incident node
                self.coord_list = np.vstack((self.coord_list, pos[self.incident]))
            else:
                # get first node of shortest alternate path
                next_node = self.get_next_node()
                # get info from travelled path if not already collected
                self.update_info(next_node) # does this and the bottom one work the first step, getting info updating pheremones before reaching the end of edge? what about incident node?
                # update pheremones
                self.update_pheremones(next_node) 
                # get coordinate steps to next node
                self.coord = self.get_steps(next_node)
                # update current node
                self.current_node = next_node
                # update current position 
                self.pos = self.coord[0]
                # add current position to full coords list
                self.coord_list = np.vstack((self.coord_list, self.pos))
        else:
            # update to next position
            self.pos = self.coord[np.where(self.pos == self.coord)[0][0] + 1]
            # add next position to full coords list
            self.coord_list = np.vstack((self.coord_list, self.pos))
    
    def at_incident(self):
        # whether at incident node
        return self.current_node == self.incident
    
    def at_next_node(self):
        # whether at next node
        return np.array_equal(self.pos, self.coord[-1])

    def get_next_node(self):
        # find shortest alternate path
        path = nx.astar_path(self.world, self.current_node, self.incident, heuristic=self.dist, weight=self.cost_func)
        return path[1]
    
    def cost_func(self, u, v, e):
        # get distance of edge
        cost = e['dist']
        # if pheremone exists
        if e['cost'] >= 1:
            # penalise edge
            cost *= self.cost
        return cost
    
    def dist(self, a, b):
        return math.dist(self.world_pos[a], self.world_pos[b])
    
    def get_steps(self, next_node):
        # get evenly spaced intervals between node
        steps = np.linspace(0, 1, self.world[self.current_node][next_node]['dist'])
        return self.get_distance(steps, next_node)
    
    def get_distance(self, steps, next_node):
        # get positions of intervals
        return steps[..., None] * pos[next_node] + (1 - steps[..., None]) * pos[self.current_node]
    
    def update_pheremones(self, next_node):
        # update pheromones on the edges visited
        self.world.edges[self.current_node, next_node]['cost'] += 1
        
    def update_info(self, next_node):
        # edge colour
        edge_color = self.world.edges[self.current_node, next_node]['color']
        if edge_color != 'k':
            if edge_color[-1] != 1.0:
                updated_color = edge_color[:-1] + (edge_color[-1] + 0.25,)
                self.world.edges[self.current_node, next_node]['color'] = updated_color
        else:
            self.world.edges[self.current_node, next_node]['color'] = (0.6, 0.0, 1.0, 0.25)
        
        # point info
        if self.world.edges[self.current_node, next_node]['info'] == 1:
            self.world.edges[self.current_node, next_node]['info'] = 0
            self.bank += 1

## The Network

The `World` manager.

In [ ]:
np.random.seed(17)
def voronoi_to_networkx(points):
    vor = Voronoi(points)
    G = nx.Graph()
    for simplex in vor.ridge_vertices:
        if -1 not in simplex:
            i, j = simplex
            p = vor.vertices[i]
            q = vor.vertices[j]
            if all(0 <= x <= 1 for x in p) and all(0 <= x <= 1 for x in q):
                distance = np.linalg.norm(p - q)
                G.add_edge(tuple(p), tuple(q), weight=distance)
    pos = {i: node for i, node in enumerate(G.nodes)}
    G = nx.convert_node_labels_to_integers(G, first_label=0, ordering='default')
    return G, pos

# add randomness
points = np.random.rand(200, 2)

# add non-uniformity
cluster_means = [[0.4, 0.4], [0.8, 0.8]] # where the clusters appear
cluster_stds = [0.05, 0.05] # how tight should the clustering be
for mean, std in zip(cluster_means, cluster_stds):
    cluster = np.random.normal(loc=mean, scale=std, size=(100, 2)) # generater random points for those clusters
    points = np.concatenate([points, cluster]) # add points to full points ilst

G, pos = voronoi_to_networkx(points)
    
def distance(u, v, pos):
    x1, y1 = pos[u]
    x2, y2 = pos[v]
    return math.ceil(math.sqrt((x2 - x1)**2 + (y2 - y1)**2) * 100)

for u, v in G.edges():
    G.edges[u,v]['dist'] = distance(u, v, pos)

pheromones = {(u, v): {'cost': 0} for u, v in G.edges()}
information = {(u, v): {'info': 1} for u, v in G.edges()}
nx.set_edge_attributes(G, pheromones)
nx.set_edge_attributes(G, information)

In [ ]:
class World:
    
    def __init__(self, world, decay):
        """ Initalise world attributes. """
        self.world = world
        nx.set_edge_attributes(self.world, 'k', 'color')
        self.edge_mappings = []
        self.decay = decay
        self.incident = 224#np.random.choice(list(world.nodes()))
        
    def update_map(self):
        self.edge_mappings.append([self.world[u][v]['color'] for u, v in self.world.edges()])

## The Simulator

The `Simulator` runs the simulations.

In [ ]:
class Simulation:
    
    def __init__(self, world, pos, decay, patrols, cost):
        """ Initalise simulation parameters. """
        self.world = World(world, decay)
        self.patrols = [Patrol(world, pos, self.world.incident, cost) for i in range(patrols)]
        self.instruments = []

    def run(self):
        # initialize instruments
        self.update_instruments()
        # run until incident resolved
        while not all(patrol.resolved for patrol in self.patrols):
            self.step()
        return self.world.incident, self.get_coords(), self.world.edge_mappings

    def step(self):
        for patrol in self.patrols:
            patrol.move()
            self.update_instruments()
        self.world.update_map()
    
    def add_instrument(self, instrument):
        self.instruments.append(instrument)
    
    def update_instruments(self):
        for instrument in self.instruments:
            instrument.update(self)
        
    def plot(self, index, *args, **kwargs):
        self.instruments[index].plot(*args, **kwargs)
        
    def gather(self):
        return sum([patrol.bank for patrol in self.patrols])

    def get_coords(self):
        return [patrol.coord_list for patrol in self.patrols]

In [ ]:
np.random.seed(17)
model = Simulation(world = G,
                   pos = pos,
                   decay = 0.5,
                   patrols = 20,
                   cost = 6)

In [ ]:
incident_node, coords, edge_map = model.run()

In [ ]:
def split(coords, idx):
    # split into x and y coordinates
    return [np.array(i)[:,idx] for i in coords]

x_coords = split(coords, 0)
y_coords = split(coords, 1)
colour_map = np.repeat("green", len(G.nodes()))
node_map = np.repeat(0, len(G.nodes()))
colour_map[incident_node] = "red"
node_map[incident_node] = 50

In [ ]:
max_length = max(len(lst) for lst in coords)
max_length

In [ ]:
unique_edges = len([i for i in edge_map[-1] if i != 'k'])
print(unique_edges, len(edge_map[-1]))

In [ ]:
fig = plt.figure()

def animate(i):
    plt.cla()
    nx.draw(G, pos, node_size=node_map, node_color=colour_map, edge_color=edge_map[i])
    x = 'rs' if i % 4 == 0 else 'bs'
    for j in range(len(x_coords)):
        plt.plot(x_coords[j][i], y_coords[j][i], x, markersize=3) # just change x back to 'bs'

ani = animation.FuncAnimation(plt.gcf(), animate, frames=len(x_coords[0]), interval=50, repeat=False)
plt.show()

## test

In [ ]:
#def draw_circle(radius, colors, incident_node):
#    x, y = pos[incident_node]
#    circles = [(0.4, colors[2]), (0.2, colors[1]), (0.1, colors[0])]
#    for r, color in circles:
#        circle = plt.Circle((x, y), r, fill=True, color=color)
#        circle.set_alpha(0.3)
#        ax.add_patch(circle)

#def animate(i):
#    plt.cla()
#    nx.draw(G, pos, node_size=node_map, node_color=colour_map, edge_color=edge_map[i])
#    for j in range(len(x_coords)):
#        plt.plot(x_coords[j][i], y_coords[j][i], 'bs', markersize=3)
#    # Draw the circle on top of the graph
#    draw_circle(radius, colors, incident_node)

#fig = plt.figure()
#ax = plt.gca()  # Get the current axes

# Define your variables (colors, radius, incident_node)
#colors = ['red', 'yellow', 'green']
#radius = 0.1

#ani = animation.FuncAnimation(fig, animate, frames=len(x_coords[0]), interval=50, repeat=False)

#plt.show()

In [ ]:
#ani.save(r"C:\Users\Mitch\OneDrive\Documents\MASTER OF DATA SCIENCE\SEM 1 2023\(CITS5014 and CITS5015) Data Science Research Project/animation_test.gif")

## Instruments

These `Instruments` compute metrics at each timestep.

In [ ]:
class Instrument:
    
    def __init__(self):
        self.metrics = []
        
    def update(self, model):
        pass
        
    def plot(self, **options):
        plt.plot(self.metrics, **options) # do I need this?

In [ ]:
class Gather(Instrument):
    """Computes information gathered at each timestep."""
    label = 'Information Gathered'
    
    def update(self, model):
        info_count = model.gather()
        self.metrics.append(info_count)

In [ ]:
model = Simulation(world = G,
                   pos = pos,
                   decay = 0.5,
                   patrols = 15,
                   cost = 1)
instrument = Gather()
model.add_instrument(instrument)
model.run()
model.plot(index=0)

def label(xlabel, ylabel, graph):
    plt.xlabel(xlabel)
    plt.ylabel(ylabel.format(len(graph.edges())))
    plt.show()

label('Time', 'Info gathered/unique edges traversed: {} total edges', G)